In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
# Load the dataset
iris = load_iris()
df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
df['target'] = iris.target
df.sample(7)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
99,5.7,2.8,4.1,1.3,1
133,6.3,2.8,5.1,1.5,2
127,6.1,3.0,4.9,1.8,2
111,6.4,2.7,5.3,1.9,2
78,6.0,2.9,4.5,1.5,1
95,5.7,3.0,4.2,1.2,1
30,4.8,3.1,1.6,0.2,0


In [3]:
# Split the data
train,test = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
train_X = train.drop(["target"],axis=1).values
train_y = train[["target"]].values.ravel()

# Test dataset
test_X = test.drop(["target"],axis=1).values
test_y = test[["target"]].values.ravel()

## Spliting into training and validation set
train_X,valid_X,train_y,valid_y = train_test_split(train_X,train_y,test_size=0.2,random_state=42)
signature = infer_signature(train_X,train_y)

In [5]:
print("train_X shape:", train_X.shape)
print("train_y shape:", train_y.shape)
print("valid_X shape:", valid_X.shape)
print("valid_y shape:", valid_y.shape)

train_X shape: (96, 4)
train_y shape: (96,)
valid_X shape: (24, 4)
valid_y shape: (24,)


In [6]:
def train_model(params, epochs, train_X, train_y, valid_X, valid_y, test_X, test_y):
    ## define model architecture
    mean = np.mean(train_X, axis=0)
    var = np.var(train_X, axis=0)

    model = keras.Sequential([
        keras.Input(shape=(train_X.shape[1],)),  # Input layer
        keras.layers.Normalization(mean=mean, variance=var),  # Normalization layer
        keras.layers.Dense(64, activation="relu"),  # Hidden layer
        keras.layers.Dropout(0.2),  # Dropout layer
        keras.layers.Dense(3, activation="softmax")  # Output layer for 3 classes
    ])

    ## Compile the Model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    ## Train the model
    with mlflow.start_run(nested=True):
        early_stopping = keras.callbacks.EarlyStopping(patience=5, min_delta=0.001)
        model.fit(train_X, train_y, validation_data=(valid_X, valid_y),
                  epochs=epochs, verbose=2, batch_size=64, callbacks=[early_stopping])

        ## Evaluate the model
        eval_result = model.evaluate(valid_X, valid_y, batch_size=64)

        eval_rsme = eval_result[1]

        ## Log the paramter and mertic
        mlflow.log_params(params)
        mlflow.log_metric("eval_rsme",eval_rsme)

        ## Log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rsme, "status": STATUS_OK, "model": model}

In [7]:
def objective(params):
    ## MLFLOW will track the parameters and results for each run
    result = train_model(
        params,
        epochs=10,
        train_X = train_X,
        train_y = train_y,
        valid_X = valid_X,
        valid_y = valid_y,
        test_X = test_X,
        test_y = test_y
    )
    return result

In [8]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [32]:
mlflow.set_experiment("/iris-dataset")
with mlflow.start_run():
    ## Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    ## Fetch the details of best run
    best_run = min(trials.results, key=lambda x: x["loss"])

    ## Log the best parameter and metric
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    ## Print the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/10                                           

2/2 - 2s - 991ms/step - accuracy: 0.5833 - loss: 0.9675 - val_accuracy: 0.5417 - val_loss: 1.0273

Epoch 2/10                                           

2/2 - 0s - 37ms/step - accuracy: 0.6042 - loss: 0.9506 - val_accuracy: 0.5417 - val_loss: 1.0021

Epoch 3/10                                           

2/2 - 0s - 33ms/step - accuracy: 0.6250 - loss: 0.9237 - val_accuracy: 0.5417 - val_loss: 0.9775

Epoch 4/10                                           

2/2 - 0s - 50ms/step - accuracy: 0.6562 - loss: 0.9082 - val_accuracy: 0.5417 - val_loss: 0.9539

Epoch 5/10                                           

2/2 - 0s - 39ms/step - accuracy: 0.6771 - loss: 0.8587 - val_accuracy: 0.5417 - val_loss: 0.9314

Epoch 6/10                                           

2/2 - 0s - 42ms/step - accuracy: 0.6667 - loss: 0.8631 - val_accuracy: 0.5417 - val_loss: 0.9098

Epoch 7/10                                           

2/2 - 0s - 41ms/step - acc

In [ ]:
## Inferencing
from mlflow.models import validate_serving_input

model_uri = 'runs:/1a817c13f44b43bd87486ff2f0b7bd16/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_X)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

C:\Users\rahul\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([[0.30246523, 0.3958315 , 0.30170327],
       [0.33305442, 0.44655114, 0.22039445],
       [0.30716395, 0.54504687, 0.1477891 ],
       [0.313655  , 0.36751723, 0.3188278 ],
       [0.30721322, 0.46564025, 0.22714655],
       [0.34133014, 0.38921976, 0.2694501 ],
       [0.32160038, 0.3580861 , 0.3203135 ],
       [0.35896394, 0.41848826, 0.22254783],
       [0.2754654 , 0.51451886, 0.21001577],
       [0.31039912, 0.3959512 , 0.29364967],
       [0.3481709 , 0.3810048 , 0.27082428],
       [0.27988234, 0.46616974, 0.2539479 ],
       [0.3415037 , 0.4137254 , 0.24477085],
       [0.29764667, 0.44247591, 0.25987738],
       [0.28801137, 0.46080336, 0.25118533],
       [0.3488896 , 0.3793927 , 0.2717177 ],
       [0.33902252, 0.39401528, 0.26696223],
       [0.29686475, 0.43437576, 0.2687595 ],
       [0.29649723, 0.38155317, 0.32194966],
       [0.32689613, 0.41218448, 0.26091936],
       [0.2781638 , 0.456048  , 0.26578823],
       [0.32350665, 0.36728978, 0.3092035 ],
       [0.